# Advent of Code 2023 - Day 5

In [129]:
import java.io.File
import java.util.Scanner

data class Range(
    val source: Long,
    val destination: Long,
    val length: Long,
)

data class Mapping(
    val source: String,
    val destination: String,
    val ranges: List<Range>,
)

val SOURCE_TO_DESTINATION_REGEX = """(.+)-to-(.+)\s""".toRegex()
val DIGIT_REGEX = """(\d+)""".toRegex()

val (seeds, almanac) = Scanner(File("Day05.input.txt"))
    .apply { useDelimiter("\n\n") }
    .use { scanner ->
        generateSequence { if (scanner.hasNext()) scanner.next() else null }
            .mapIndexed { index, maps ->
                if (index == 0) {
                    DIGIT_REGEX.findAll(maps).map(MatchResult::value).map(String::toLong).toList()
                }
                else {
                    maps.split(":\n").let { (category, conversions) ->
                        val (source, destination) = SOURCE_TO_DESTINATION_REGEX.find(category)!!.destructured
                        Mapping(
                            source = source,
                            destination = destination,
                            ranges = conversions.split("\n")
                                .map { 
                                    DIGIT_REGEX.findAll(it).map(MatchResult::value).toList().let { (destNum, sourceNum, length) ->
                                        Range(
                                            source = sourceNum.toLong(),
                                            destination = destNum.toLong(),
                                            length = length.toLong(),
                                        )
                                    }
                                }
                        )
                    }
                }
            }
            .toList()
            .let { it.first() as List<Long> to it.drop(1) as List<Mapping> }
    }
    

### Notes

TODO: Clean up
(I'm never coming back to this.)

## Part 1

This one is really long, read the original problem.

In [130]:
operator fun Range.get(value: Long): Long? =
    takeIf { value in source..<(source + length) }?.let { destination - source + value }

operator fun Mapping.get(source: Long): Long = ranges.firstNotNullOfOrNull { it[source] } ?: source

fun List<Mapping>.map(value: Long): Long = fold(value) { acc, mapping -> mapping[acc] }

seeds.minOf { almanac.map(it) }

662197086

### Notes

Breaking down the problem into more digestible components made this a lot easier. First figure out how to map a single range, and then a single mapping, and then a list of mappings.

## Part 2

Read original problem.

In [131]:
seeds.chunked(2).map { (start, length) -> start..<(start + length) }
    .minOf {
        it.minOf { almanac.map(it) }
    }

52510809

### Notes

I just ran this for about 10 minutes or so and it worked 🙈 Some other ways I considered:
- Figure out how to "flatten" the mappings by determining how a range of mappings map to each other. i.e. Applying a source range should generate new ranges, if you "map the mappings" then you don't need to "drill down" for each number.
- Since we're looking for the lowest number, we can go "in reverse". Assume 0 is the answer and map the number backwards, if it's in the original set then that's your answer. If not, increment by 1 until you find it. You could probably binary search the lowest value too?